# Scratchpad 4 - Instruction Fine-tuning (SFT)

Why instruction fine-tuning? A pretrained LLM is only good at text completion (predicting the next token), not good at following instructions. With instruction fine-tuning, we teach LLM to better follow instructions (generating texts that are desirable responses to the user's instructions). This is often called "supervised fine-tuning (SFT)".

For SFT, we need to
1. prepare instruction fine-tuning data that has desired input output pairs
2. load and fine-tune a pretrained LLM
3. evaluate the fine-tuned LLM

## 1. Data preparation

To prepare data for supervised instruction fine-tuning, we need to:
1. Get the raw dataset and format it according to the SFT template
2. Create an SFT DataSet that formats json data into prompt template and tokenizes the texts into token IDs 
3. Create the DataLoader for the custom SFT DataSet which uses a custom collate function to prepare input and target batches of the encoded data

### 1.1 SFT Data

#### 1.1.1 Train and test data

The training data is stored in a JSON file in `./data/sft/train`. Each sample JSON data has three fields:  `instruction`, `input` , and `output`.

```json
[
    {
        "instruction": "Evaluate the following phrase by transforming it into the spelling given.",
        "input": "freind --> friend",
        "output": "The spelling of the given phrase \"freind\" is incorrect, the correct spelling is \"friend\"."
    },
    ...
]
```

The test data for evaluation later will add the fine-tuned model's response for comparison. The file will be prepared in `./data/sft/test` and each data record will have an additional `response` field, e.g.:

```json
[
    {
        "instruction": "Rewrite the sentence using a simile.",
        "input": "The car is very fast.",
        "output": "The car is as fast as lightning.",
        "model_response": "The car is as fast as a bullet."
    },
    ...
]
```

#### 1.1.2 LLM prompt templates

Now we prepare the train data into inputs to the LLM. 

There are two example prompt template formats:

##### Alpaca
which Stanford CRFM used to train [Stanford Alpaca](https://crfm.stanford.edu/2023/03/13/alpaca.html)
  - They released 52K instruction-following data and the generation script in their [repo](https://github.com/tatsu-lab/stanford_alpaca#data-release). Our training data JSON is in exactly the same JSON format.
  - The LLM prompt template reflects this format:

With the input field:

```
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
```
Without the input field:
```
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
```


##### Phi-3

which Microsoft Research used to train [Phi-3](https://github.com/microsoft/Phi-3CookBook/tree/main). 

The Phi-3 training data prompt template is as follows:

```
<|system|>
Your Role<|end|>
<|user|>
Your Question?<|end|>
<|assistant|>
```

A `jsonl` data would be like follows:

```json
{"text": "<|user|>\nWhen were iron maidens commonly used? <|end|>\n<|assistant|> \nIron maidens were never commonly used <|end|>"}
```

But when using Azure AI to fine-tune, the data format is aligned with GPT, i.e.:

```json
{"messages": [{"role": "system", "content": "You are an Xbox customer support agent whose primary goal is to help users with issues they are experiencing with their Xbox devices. You are friendly and concise. You only provide factual answers to queries, and do not provide answers that are not related to Xbox."}, {"role": "user", "content": "Is Xbox better than PlayStation?"}, {"role": "assistant", "content": "I apologize, but I cannot provide personal opinions. My primary job is to assist you with any issues related to your Xbox device. Do you have any Xbox-related issues that need addressing?"}]}
```

This notebook uses the Alpaca prompt template.

#### 1.1.3 Load and format data

Load the data from JSON file.

In [16]:
# load the training data

import json
data_file = './data/sft/train/instruction-data.json'
with open(data_file, "r", encoding="utf-8") as f:
    data = json.load(f)

In [14]:
print(
f"""number of training data JSON lines: {len(data)}
Example:\n{json.dumps(data[0], indent=4)}"""
)

number of training data JSON lines: 1100
Example:
{
    "instruction": "Evaluate the following phrase by transforming it into the spelling given.",
    "input": "freind --> friend",
    "output": "The spelling of the given phrase \"freind\" is incorrect, the correct spelling is \"friend\"."
}


Format it according to prompt template.

In [17]:
# format the data into prompt template for input to LLM

def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task."
        f"Write a response that appropriately completes the task."
        f"\n\n### Instruction\n{entry['instruction']}"
    )
    input_text = f"\n\n### Input\n{entry['input']}" if entry["input"] else ""
    return instruction_text + input_text

In [18]:
model_input = format_input(data[0])
desired_output = f"\n\n### Response:\n{data[0]['output']}"

print(model_input + desired_output)

Below is an instruction that describes a task.Write a response that appropriately completes the task.

### Instruction
Evaluate the following phrase by transforming it into the spelling given.

### Input
freind --> friend

### Response:
The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".


#### 1.1.4 Split training, validation and test datasets

In [20]:
train_portion = int(len(data) * 0.85) # 85% of the data for training
test_portion = int(len(data) * 0.1) # 10% of the data for testing
val_portion = len(data) - train_portion - test_portion # 5% of the data for validation

train_data = data[:train_portion]
test_data = data[train_portion:train_portion + test_portion]
val_data = data[train_portion + test_portion:]

In [21]:
print(f"train: {len(train_data)}, test: {len(test_data)}, val: {len(val_data)}")

train: 935, test: 110, val: 55


### 1.2 Create custom DataSet

Implement a custom DataSet that at initialization: 
- format json data into LLM prompt template
- tokenize the text into token IDs

In [22]:
import torch
from torch.utils.data import Dataset


class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.encoded_texts = []

        for entry in data:
            # format the data input and response into prompt template
            instruction_plus_input = (
                f"Below is an instruction that describes a task."
                f"Write a response that appropriately completes the task."
                f"\n\n### Instruction\n{entry['instruction']}"
            ) + (f"\n\n### Input\n{entry['input']}" if entry["input"] else "")
            response_text = f"\n\n### Response:\n{entry['output']}"
            full_text = instruction_plus_input + response_text

            # tokenize the full text
            self.encoded_texts.append(
                tokenizer.encode(full_text)
            )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.encoded_texts[idx]

Test it out.

In [40]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
train_dataset = InstructionDataset(train_data, tokenizer)

print("######## training data json ########")
print(f"training json data length: {len(train_dataset)}")
print(f"first training data json: {json.dumps(train_data[0], indent=4)}")
print("\n######## training dataset ########")
print(f"training dataset size: {len(train_dataset)}")
print(f"first item of the training dataset: \n{train_dataset[0]}")
print(f"Decoded first item of the training dataset: \n{tokenizer.decode(train_dataset[0])}")

######## training data json ########
training json data length: 935
first training data json: {
    "instruction": "Evaluate the following phrase by transforming it into the spelling given.",
    "input": "freind --> friend",
    "output": "The spelling of the given phrase \"freind\" is incorrect, the correct spelling is \"friend\"."
}

######## training dataset ########
training dataset size: 935
first item of the training dataset: 
[21106, 318, 281, 12064, 326, 8477, 257, 4876, 13, 16594, 257, 2882, 326, 20431, 32543, 262, 4876, 13, 198, 198, 21017, 46486, 198, 36, 2100, 4985, 262, 1708, 9546, 416, 25449, 340, 656, 262, 24993, 1813, 13, 198, 198, 21017, 23412, 198, 19503, 521, 14610, 1545, 198, 198, 21017, 18261, 25, 198, 464, 24993, 286, 262, 1813, 9546, 366, 19503, 521, 1, 318, 11491, 11, 262, 3376, 24993, 318, 366, 6726, 1911]
Decoded first item of the training dataset: 
Below is an instruction that describes a task.Write a response that appropriately completes the task.

### Inst

### 1.3 Create DataLoader with a custom colloate function

## 2. Finetuning a pretrained LLM

To fine-tune a pretrained LLM, we need to
1. load the pretrained LLM
2. train (fine-tune) it on SFT data
3. save the model

## 3. Evaluating the fine-tuned LLM

To evaluate the fine-tuned LLM, we will use another LLM.
1. run inference on test set and save the responses
2. compare ground truth in test set with generated responses
3. use another LLM to score the fine-tuned responses